In [1]:
import numpy as np
import cv2
import multiprocessing
import matplotlib.pyplot as plt
import time
from my_args import MyArgs
from video_extracter import VideoExtracter
import pickle

In [2]:
def get_output_dim(size, h, w):
        if isinstance(size, tuple) and len(size) == 2:
            return size
        elif h >= w:
            return int(h * size / w), size
        else:
            return size, int(w *size / h)

In [3]:
args = MyArgs(model_type='3d',batch_size = 10,framerate=30,scale_time=2)

In [4]:
v_extracter = VideoExtracter(args)

Loading 3D-ResneXt-101 ...
loaded


***

In [5]:
def start_camera(model_type,time_split,v_extracter_,clf_sknn,clf_knn,clf_svm,clf_ada):
    cap = cv2.VideoCapture(0)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    weight = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    h, w = get_output_dim(112,height,weight)
    delays = []
    video = []
    warningflag_sknn = False
    warningflag_knn = False
    warningflag_svm = False
    warningflag_ada = False
    # while (countstop < frame_rate*5):
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()        
        # Our operations on the frame come here
        if model_type == '3d':
            vframe = cv2.resize(frame[:,::-1,:],(112,112),fx=0,fy=0, interpolation = cv2.INTER_AREA)
        elif model_type == '2d':
            vframe = cv2.resize(frame[:,::-1,:],(w,h),fx=0,fy=0, interpolation = cv2.INTER_AREA)
        else:
            print('Model type must be: 2d or 3d')
            break        
        video.append(vframe)
        if len(video) == frame_rate * time_split:
            temp = video.copy()
            video.clear()
            start = time.time()
            feature = v_extracter_.predict(temp)
            feature = feature.flatten().reshape(1,-1)
            pred_sknn = clf_sknn.predict(feature)
            pred_knn = clf_knn.predict(feature)
            pred_svm = clf_svm.predict(feature)
            pred_ada = clf_ada.predict(feature)
            if pred_sknn == 1:
                warningflag_sknn = True
            else:
                warningflag_sknn = False
            if pred_knn == 1:
                warningflag_knn = True
            else:
                warningflag_knn = False
            if pred_svm == 1:
                warningflag_svm = True
            else:
                warningflag_svm = False
            if pred_ada == 1:
                warningflag_ada = True
            else:
                warningflag_ada = False
            end = time.time()
            delays.append(end - start)
        img_sknn = np.array(frame[:,::-1,:])
        img_knn = np.array(frame[:,::-1,:])
        img_svm = np.array(frame[:,::-1,:])
        img_ada = np.array(frame[:,::-1,:])
        if warningflag_sknn:
            img_sknn = cv2.putText(img_sknn, 'Warnning: fighting detected', (20,30), cv2.FONT_HERSHEY_SIMPLEX , 0.5, (0,0,255), 1, cv2.LINE_AA)
        if warningflag_knn:
            img_knn = cv2.putText(img_knn, 'Warnning: fighting detected', (20,30), cv2.FONT_HERSHEY_SIMPLEX , 0.5, (0,0,255), 1, cv2.LINE_AA)
        if warningflag_svm:
            img_svm = cv2.putText(img_svm, 'Warnning: fighting detected', (20,30), cv2.FONT_HERSHEY_SIMPLEX , 0.5, (0,0,255), 1, cv2.LINE_AA)
        if warningflag_ada:
            img_ada = cv2.putText(img_ada, 'Warnning: fighting detected', (20,30), cv2.FONT_HERSHEY_SIMPLEX , 0.5, (0,0,255), 1, cv2.LINE_AA)
        # Display the resulting frame
        cv2.imshow('Smote KNN',img_sknn)
        cv2.imshow('KNN',img_knn)
        cv2.imshow('SVM Poly',img_svm)
        cv2.imshow('Adaboost',img_ada)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()
    return delays

In [6]:
# You change model's absolute path to your path
clf_sknn = pickle.load(open('./model_ml/model_with_smote/smote_01_knn_n17.sav','rb'))
clf_knn = pickle.load(open('./model_ml/model/01_knn_n17.sav', 'rb'))
clf_svm = pickle.load(open('./model_ml/model/01_svm_poly.sav', 'rb'))
clf_ada = pickle.load(open('./model_ml/model/01_adaboost.sav', 'rb'))

In [7]:
delays = start_camera('3d',args.scale_time,v_extracter,clf_sknn,clf_knn,clf_svm,clf_ada)

In [18]:
#delays

In [9]:
np.mean(delays[1:])

0.21478606933771177